In [1]:
import sqlite3
import pandas as pd
import filler
import trainer

# Check data quality

In [3]:
def identify_data_issues(df):
    """
    Identify duplicated timestamps, missing intervals, and 
    out-of-order rows in a time-indexed DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing a 'Time' column.
        
    Returns:
        duplicates (pd.DataFrame): Rows that contain duplicate timestamps.
        missing_intervals (pd.DatetimeIndex): Start points of intervals where data is missing.
        is_unordered (bool): True if the data is not strictly time-ordered.
    """
    # Ensure 'Time' is a datetime column
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    
    # Check for any NaT (Not a Time) conversions which indicate invalid timestamps
    if df['Time'].isnull().any():
        raise ValueError("Some timestamps could not be converted to datetime.")
    
    # Check if the data is ordered by 'Time'. If not, sort it.
    if not df['Time'].is_monotonic_increasing:
        is_unordered = True
        df = df.sort_values('Time').reset_index(drop=True)
    else:
        is_unordered = False
    
    # Identify duplicates
    duplicates = df[df.duplicated(subset='Time', keep=False)]

    # Identify missing intervals
    # Calculate the difference between consecutive rows
    time_diffs = df['Time'].diff().dropna()
    # Any gap larger than 1 minute indicates missing intervals
    missing_intervals = time_diffs[time_diffs > pd.Timedelta('1min')]

    return duplicates, missing_intervals, is_unordered

# Example usage:
df = filler.read_data_from_table("ethusd")  # Assuming you have a function that reads from SQLite
duplicates, missing_intervals, is_unordered = identify_data_issues(df)
print("Duplicates:\n", duplicates)
print("Missing intervals:\n", missing_intervals)
print("Data is unordered:", is_unordered)


Duplicates:
 Empty DataFrame
Columns: [Time, Open, High, Low, Close, Volume]
Index: []
Missing intervals:
 Series([], Name: Time, dtype: timedelta64[ns])
Data is unordered: False


# See database

In [26]:
# Define database connection function
def connect_to_sqlite():
    conn = sqlite3.connect("crypto_data.sqlite", check_same_thread=False)
    conn.execute("PRAGMA journal_mode=WAL;")  # Enable concurrent reads and writes
    return conn

def load_preds():
    conn = connect_to_sqlite()
    query = """
        SELECT * 
        FROM predictions 
        WHERE timestamp > '2024-12-18 18:55:00'
        ORDER BY timestamp DESC 
        """
    df = pd.read_sql_query(query, conn)  # Read the data into a pandas DataFrame
    conn.close()
    return df.sort_values('timestamp')

preds = load_preds()
# Remove duplicate rows from the DataFrame
preds = df.drop_duplicates()

def load_data():
    conn = connect_to_sqlite()
    query = """
        SELECT * 
        FROM ethusd 
        WHERE Time > '2024-12-18 16:55:00'
        ORDER BY Time DESC 
        """
    df = pd.read_sql_query(query, conn)  # Read the data into a pandas DataFrame
    conn.close()
    return df.sort_values('Time')

data = load_data()
# Remove duplicate rows from the DataFrame
data = data.drop_duplicates()

preds

,timestamp,current_close_price,predicted_next_absolute_max,predicted_percentage_change
117,2024-12-18 18:54:00,3885.71,3686.692627,-5.121776
116,2024-12-18 18:55:00,3885.28,3686.484619,-5.116629
115,2024-12-18 18:57:00,3885.29,3883.696533,-0.041013
114,2024-12-18 18:58:00,3890.19,3885.306152,-0.125543
112,2024-12-18 18:59:00,3894.80,3886.915283,-0.202442
...,...,...,...,...
4,2024-12-18 20:44:00,3678.25,3668.653809,-0.260890
3,2024-12-18 20:45:00,3683.36,3661.597900,-0.590822
2,2024-12-18 20:46:00,3695.07,3662.131836,-0.891408
1,2024-12-18 20:47:00,3687.04,3664.240479,-0.618369


In [27]:
# Convert to datetime if not already
preds['timestamp'] = pd.to_datetime(preds['timestamp'])
data['Time'] = pd.to_datetime(data['Time'])

# Create a new column in preds for the join key: timestamp minus 60 minutes
preds['timestamp_minus_60'] = preds['timestamp'] - pd.Timedelta(minutes=60)

# Perform a left merge, matching preds.timestamp_minus_60 to data.Time
merged_df = preds.merge(
    data[['Time', 'Close']],            # Select only the columns we need from data
    how='left',
    left_on='timestamp_minus_60',
    right_on='Time'
)

# The merged_df now contains all rows from preds along with the matching 'Close' from data 
# where data.Time = preds.timestamp - 60 minutes.

# You can drop the 'Time' column (from data) and the helper column if you like:
merged_df = merged_df.drop(columns=['Time', 'timestamp_minus_60'])

merged_df


/var/folders/q5/98_dfqbd17gg8kthxrn22rph0000gn/T/ipykernel_33730/2089305636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preds['timestamp'] = pd.to_datetime(preds['timestamp'])
/var/folders/q5/98_dfqbd17gg8kthxrn22rph0000gn/T/ipykernel_33730/2089305636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preds['timestamp_minus_60'] = preds['timestamp'] - pd.Timedelta(minutes=60)


,timestamp,current_close_price,predicted_next_absolute_max,predicted_percentage_change,Close
0,2024-12-18 18:54:00,3885.71,3686.692627,-5.121776,3882.97
1,2024-12-18 18:55:00,3885.28,3686.484619,-5.116629,3883.60
2,2024-12-18 18:57:00,3885.29,3883.696533,-0.041013,3878.72
3,2024-12-18 18:58:00,3890.19,3885.306152,-0.125543,3878.59
4,2024-12-18 18:59:00,3894.80,3886.915283,-0.202442,3878.58
...,...,...,...,...,...
109,2024-12-18 20:44:00,3678.25,3668.653809,-0.260890,3841.41
110,2024-12-18 20:45:00,3683.36,3661.597900,-0.590822,3833.83
111,2024-12-18 20:46:00,3695.07,3662.131836,-0.891408,3823.39
112,2024-12-18 20:47:00,3687.04,3664.240479,-0.618369,3813.40


# Run trainer

In [2]:
from trainer import train_model

# Custom hyperparameters
custom_hyperparameters = {
    "n_estimators": 200,
    "max_depth": 3,
    "learning_rate": 0.1,
    "min_child_weight": 5,
    "colsample_bytree": 0.5,
    "tree_method": 'hist',
}

train_model(hyperparameters=custom_hyperparameters)

Starting model training...
Model training completed and model saved.
